In [1]:
# Import Dependencies and modules
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from string import punctuation
from collections import Counter
from io import StringIO
from nltk.corpus import stopwords
import nltk
import glob
import errno
import os
import json

# Load Data

In [2]:
json_data = pd.DataFrame(columns = ['author', 'text', 'title'])

In [3]:
# Define path to text files that contain the novels
path = '/Users/ayankarim/Documents/Thinkful/Bootcamp/Final Capstone Opinion Mining/opinion_mining/Notebook/Topic Modelling Working Set/files/*.json'
files = glob.glob(path)

all_jsons = []

for filenames in files:
    with open(filenames, 'r') as f:
        file = json.load(f)
        all_jsons.append(file)


In [4]:
for dicts in all_jsons:
    df = pd.DataFrame(dicts)
    json_data = json_data.append(df)

In [5]:
json_data.head()

,author,text,title
0,[\n\t\t\t\t\t\tJulian Chokkattu\t\t\t\t\t],"[<p>Google’s <a href=""https://store.google.com...",\n\t\tGoogle will announce hardware on October...
1,[\n\t\t\t\t\t\tChristian de Looper\t\t\t\t\t],"[<p>Google finally unveiled the new <a href=""h...",\n\t\tHere’s how to buy the new Google Pixel 3...
2,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],[<p>If you plan to buy one of Google’s <a href...,\n\t\tThe best Pixel 3 cases and covers\t
3,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],"[<p>As the developer of Android, Google turns ...",\n\t\tGoogle Pixel 3 vs. Pixel 2 vs. Pixel: Pi...
4,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],[<p>There are plenty of contenders in the <a h...,\n\t\tGoogle Pixel 3 vs. Samsung Galaxy S9: Wh...


# Clean Data

In [6]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!

    text = str(text).replace("\n", "")
    text = str(text).replace("\t", "")
    text = str(text).replace("\\n", "")
    text = str(text).replace("\\t", "")
    text = str(text).replace("\\", "")
    text = str(text).replace("xa0", " ")
    text = str(text).replace("\'", "")
    text = re.sub("<p>", "", str(text))
    text = re.sub("</p>", "", str(text))
    text = re.sub("</a>", "", str(text))
    text = re.sub('<[^>]+>', "", str(text)) 
    text = str(text).replace("\\u2019", "")
    text = str(text).replace("\\u2013", "")
    text = str(text).replace("\\u2018", "")
    text = str(text).replace("\\u00a0", "")
    text = str(text).replace("\\u00a3", "")
    text = str(text).replace("\u2014", "")
    text = str(text).replace("\u201d", "")
    text = str(text).replace("\u201c", "")
    return text


In [7]:
# Define function to clean text
def clean_text(df):
    # Convert lists to strings and remove brackets
    df['text'] = df['text'].astype(str)
    df['author'] = df['author'].astype(str)

    df['text'] = df['text'].map(lambda x: x.strip('[]'))
    df['author'] = df['author'].map(lambda x: x.strip('[]'))

    # Clean text
    df['text'] = df['text'].apply(lambda x: text_cleaner(x))
    df['title'] = df['title'].apply(lambda x: text_cleaner(x))
    df['author'] = df['author'].apply(lambda x: text_cleaner(x))

In [8]:
# Clean Text
clean_text(json_data)

In [9]:
# Reset index
json_data = json_data.reset_index()
json_data = json_data.drop(['index'], axis=1)

In [10]:
# Visualize dataframe
json_data

,author,text,title
0,Julian Chokkattu,Google’s annual hardware launch event will tak...,"Google will announce hardware on October 9, ne..."
1,Christian de Looper,Google finally unveiled the new Google Pixel 3...,Here’s how to buy the new Google Pixel 3 and G...
2,Simon Hill,If you plan to buy one of Google’s Pixel 3 sma...,The best Pixel 3 cases and covers
3,Simon Hill,"As the developer of Android, Google turns out ...",Google Pixel 3 vs. Pixel 2 vs. Pixel: Picking ...
4,Simon Hill,There are plenty of contenders in the Android ...,Google Pixel 3 vs. Samsung Galaxy S9: Which sm...
5,Julian Chokkattu,Got your hands on a new Pixel 3 or Pixel 3 XL ...,Key settings you need to change on your brand-...
6,Simon Hill,Rarely has a flagship phone been so thoroughly...,Google Pixel 3 and Pixel 3 XL: Everything you ...
7,Lucas Coll,"Mobile hardware is getting better and better, ...","Verizon’s buy one, get one offer is the best d..."
8,Christian de Looper,The Google Pixel 3 and Pixel 3 XL may have sto...,The Google Pixel Stand turns your Android phon...
9,Simon Hill,The Google Pixel 3 and Pixel 3 XL are phones w...,The best Google Pixel 3 tips and tricks


# Pre-Process Data for NLP

In [11]:
# # Tokenize text
# df['text'] = df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
# df['title'] = df.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)

# # Remove Stopwords, or keep it, might be important for aspect based semantics
# stop = stopwords.words('english')
# df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stop])
# df['title'] = df['title'].apply(lambda x: [item for item in x if item not in stop])

# # Lowercase everything
# df['text'] = df['text'].astype(str)
# df['text'] = df['text'].apply(lambda x: x.lower())

# df['title'] = df['title'].astype(str)
# df['title'] = df['title'].apply(lambda x: x.lower())

# # remove all punctuations
# df['text'] = df['text'].apply(lambda x: ''.join(c for c in x if c not in punctuation))
# df['title'] = df['title'].apply(lambda x: ''.join(c for c in x if c not in punctuation))

In [12]:
from spacy.lang.en import English

spacy.load('en')
parser = English()

# Function to tokenize text
def tokenize(text):
    lda_tokens = []
    tokens  = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [13]:
# Download wordnet to find meaning of words, synonyms and antonyms
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ayankarim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
from nltk.corpus import wordnet as wn

# Function to lemmatize and more words to their root
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
# Compile set of stopwords
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ayankarim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
text_data = []

# Prepare training set for LDA
tokens = json_data['text'].apply(lambda x: prepare_text_for_lda(x))

# Prepare Dataframe for later
json_data['text'] = json_data['text'].apply(lambda x: prepare_text_for_lda(x))

# Append tokenized text to list of tokenized data
null = tokens.apply(lambda x: text_data.append(x))

In [16]:
# Visualize dataframe
json_data.head()

,author,text,title
0,Julian Chokkattu,"[google, annual, hardware, launch, event, plac...","Google will announce hardware on October 9, ne..."
1,Christian de Looper,"[google, finally, unveil, google, pixel, pixel...",Here’s how to buy the new Google Pixel 3 and G...
2,Simon Hill,"[google, pixel, smartphones, better, protect, ...",The best Pixel 3 cases and covers
3,Simon Hill,"[developer, android, google, turn, slick, andr...",Google Pixel 3 vs. Pixel 2 vs. Pixel: Picking ...
4,Simon Hill,"[plenty, contender, android, heavyweight, divi...",Google Pixel 3 vs. Samsung Galaxy S9: Which sm...


In [17]:
from gensim import corpora

# Assemble tokenized text data into a dictionary
dictionary = corpora.Dictionary(text_data)

In [18]:
# Create Bag of Words corpus from text data
corpus = [dictionary.doc2bow(text) for text in text_data]

In [19]:
from gensim import models

# Create TF-IDF vectors from our bag of words
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.04858421668076515),
 (1, 0.15597581231926652),
 (2, 0.15597581231926652),
 (3, 0.05093407669952885),
 (4, 0.14408021852527553),
 (5, 0.08740684723549623),
 (6, 0.10653367633621257),
 (7, 0.09079346621782775),
 (8, 0.08519813423287882),
 (9, 0.03306174460012256),
 (10, 0.0340415226055659),
 (11, 0.03913910897692425),
 (12, 0.008823802094586674),
 (13, 0.15597581231926652),
 (14, 0.02602018120439237),
 (15, 0.04332036655419999),
 (16, 0.10228001450001582),
 (17, 0.13684898321855016),
 (18, 0.040787012377816345),
 (19, 0.10228001450001582),
 (20, 0.04247777976041215),
 (21, 0.08124938358186642),
 (22, 0.06502989167225377),
 (23, 0.04345950405360564),
 (24, 0.11772215411783378),
 (25, 0.11772215411783378),
 (26, 0.08124938358186642),
 (27, 0.029168440631058362),
 (28, 0.05008658800839868),
 (29, 0.08691900810721127),
 (30, 0.03400228155900928),
 (31, 0.15597581231926652),
 (32, 0.06502989167225377),
 (33, 0.17360667793486),
 (34, 0.0680830452111318),
 (35, 0.05093407669952885),
 (36

In [20]:
import pickle

pickle.dump(corpus_tfidf, open('corpus_tfidf.pkl', 'wb'))
dictionary.save('dictionary.gensim')

# 3 Topics

In [21]:
import gensim

NUM_TOPICS = 3

# Extract Topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics = NUM_TOPICS, id2word = dictionary, passes=50)
ldamodel.save('model3.gensim')

# Print terms for topics
topics = ldamodel.print_topics(num_words=7)
for topic in topics:
    print(topic)

(0, '0.000*"ascent" + 0.000*"subaru" + 0.000*"brainers" + 0.000*"journey" + 0.000*"invincible" + 0.000*"ubreakifix" + 0.000*"warranty"')
(1, '0.003*"pixel" + 0.003*"iphone" + 0.002*"huawei" + 0.002*"oneplus" + 0.002*"google" + 0.002*"essential" + 0.002*"samsung"')
(2, '0.000*"brainers" + 0.000*"journey" + 0.000*"invincible" + 0.000*"plane" + 0.000*"quirky" + 0.000*"outdo" + 0.000*"warranty"')


# 4 Topics

In [22]:
import gensim

NUM_TOPICS = 4

# Extract Topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics = NUM_TOPICS, id2word = dictionary, passes=50)
ldamodel.save('model4.gensim')

# Print terms for topics
topics = ldamodel.print_topics(num_words=7)
for topic in topics:
    print(topic)

(0, '0.000*"hydrogen" + 0.000*"4view" + 0.000*"nubia" + 0.000*"suspension" + 0.000*"matebook" + 0.000*"dynamix" + 0.000*"polaris"')
(1, '0.003*"pixel" + 0.003*"iphone" + 0.003*"huawei" + 0.002*"oneplus" + 0.002*"google" + 0.002*"essential" + 0.002*"samsung"')
(2, '0.000*"woman" + 0.000*"pants" + 0.000*"hillard" + 0.000*"idrop" + 0.000*"jean" + 0.000*"explode" + 0.000*"holster"')
(3, '0.000*"mclaren" + 0.000*"ossia" + 0.000*"ubreakifix" + 0.000*"warranty" + 0.000*"repair" + 0.000*"transmitter" + 0.000*"lawder"')


# pyLDAvis

In [23]:
# Create Visualizations of topic clusters using pyLDAvis

dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('/Users/ayankarim/Documents/Thinkful/Bootcamp/Final Capstone Opinion Mining/opinion_mining/Notebook/Topic Modelling Working Set/corpus_tfidf.pkl', 'rb'))


In [24]:
import pyLDAvis.gensim

# Display 3 topics
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [25]:
# Display 4 topics
lda4 = gensim.models.ldamodel.LdaModel.load('model4.gensim')
lda_display4 = pyLDAvis.gensim.prepare(lda4, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display4)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [26]:
### Scrape more Data
    # Huawei Mate 20 Pro 
    # OnePlus 6T
    # Huawei P2 Pro
    # LG V4 Thinq
    # Sony Xperia XZ3
    # Essential Phone
    # Razor Phone 2
    # HTC U12+
    # Moto G6 plus